In [ ]:
import tensorflow as tf

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2. Variable

**函数：tf.Variable(initial-value, name= optional-name)** <br>
#### 问1 为什么constant的第一个字母是小写c，而Variable的第一个字母是大写V?
&emsp; tf.constant 是一个op <br>
&emsp; tf.Variable 是一个ops <br>
- x.initializer # init op
- x.value() # read op
- x.assign(...) # write op    
- x.assign_add(...) # and more


### 2.1 Variable的声明和初始化

- Variable在使用前需要初始化操作

In [ ]:
s = tf.Variable (2, name = 'scalar')
v = tf.Variable ([1, 2, 3], name = 'vector')
m = tf.Variable (tf.random_normal([784, 10]), name = 'matrix')

with tf.Session() as sess:
    # 2.1.1 简单粗暴，初始化所有变量
    sess.run (tf.initialize_all_variables())
    # 2.1.2 只初始化部分变量
    sess.run (tf.variables_initializer([s, m]))
    # 2.1.3 只初始化一个变量m
    sess.run (m.initializer)
    print (m.eval())
#    print(sess.run(W))   >> FailedPreconditionError: Attempting to use uninitialized value Variable


## 2.2 Variable的assign操作

In [ ]:
W = tf.Variable (10, name = 'W')
assign_op = tf.assign(W, 100)

init = tf.global_variables_initializer()
# 这个会话框中因为没有run assign_op，所以这个assign函数不会调用,值不变
with tf.Session() as sess:
    sess.run (init)
    print (W.eval())

with tf.Session() as sess:
    sess.run (init)
    sess.run (assign_op)
    print (W.eval())

## 2.3 Variable的sub和add操作

In [ ]:
w_1 = tf.Variable (10, name = 'w_1')

with tf.Session() as sess:
    sess.run (tf.global_variables_initializer())
    print ("w_1 = ", w_1.eval())
    sess.run (w_1.assign_add (10))
    print ("w_1 = ", w_1.eval())
    sess.run (w_1.assign_sub (5))
    print ("w_1 = ", w_1.eval())

## 2.4 不同Session下的sub和add操作 

&emsp;&emsp;**不同Session的值是互相独立的**

In [ ]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10)))  # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8

print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50))) # >> -42

sess1.close()
sess2.close()


## tf.get_variable()
W = tf.get_variable(name, shape=None, dtype=tf.float32, initializer=None,
       regularizer=None, trainable=True, collections=None) <br>
**相较于tf.Variable推荐使用这个，因为tf.get_variable() 会检查当前命名空间下是否存在同样name的变量，可以方便共享变量。而tf.Variable 每次都会新建一个变量。**<br>
**需要注意的是tf.get_variable() 要配合reuse和tf.variable_scope() 使用。** <br>
**再次运行的时候需要restart**

In [ ]:
with tf.variable_scope("ones"):
    a = tf.get_variable("v", [1]) #a.name == "one/v:0"
with tf.variable_scope("ones", reuse = True): #注意reuse的作用。
    c = tf.get_variable("v", [1]) #c.name == "one/v:0" 成功共享，因为设置了reuse
with tf.variable_scope("ones"):
    b = tf.get_variable("v", [1]) #创建两个名字一样的变量会报错 ValueError: Variable one/v already exists 
assert a==c #Assertion is true, they refer to the same object.

In [ ]:
with tf.variable_scope("two"):
    d = tf.get_variable("v", [1]) #d.name == "two/v:0"
    e = tf.Variable(1, name = "v", expected_shape = [1]) #e.name == "two/v_1:0"  

assert d==e #AssertionError: they are different objects